<h1 align="center">Module 4 Assessment</h1>

## Overview

This assessment is designed to test your understanding of the Mod 4 material. It covers:

* Calculus, Cost Function, and Gradient Descent
* Extensions to Linear Models
* Introduction to Linear Regression
* Time Series Modeling


Read the instructions carefully. You will be asked both to write code and respond to a few short answer questions.

### Note on the short answer questions

For the short answer questions please use your own words. The expectation is that you have not copied and pasted from an external source, even if you consult another source to help craft your response. While the short answer questions are not necessarily being assessed on grammatical correctness or sentence structure, do your best to communicate yourself clearly.

---
## Calculus, Cost Function, and Gradient Descent
---

![best fit line](visuals/best_fit_line.png)

The best fit line that goes through the scatterplot up above can be generalized in the following equation: $$y = mx + b$$

Of all the possible lines, we can prove why that particular line was chosen using the plot down below:

![](visuals/cost_curve.png)

where RSS is defined as the residual sum of squares:

$$ 
\begin{align}
RSS &= \sum_{i=1}^n(actual - expected)^2 \\
&= \sum_{i=1}^n(y_i - \hat{y})^2 \\
&= \sum_{i=1}^n(y_i - (mx_i + b))^2
\end{align}
$$ 

### 1. What is a more generalized name for the RSS curve up above?

// your answer here //

### 2A. Would you rather choose a $m$ value of 0.08 or 0.03 from the curve up above? 

// your answer here //

### 2B. Explain what it means to move along the curve in relation to the best fit line with respect to the $m$ value you chose in 2A.

// your answer here //

![](visuals/gd.png)

### 3. Using the gradient descent visual from above, explain why the distance between steps is getting smaller.

// your answer here //

### 4. Explain how a very small and a very large learning rate would affect the gradient descent.

// your answer here //

---
## Extensions to Linear Regression
---

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.linear_model import Lasso, Ridge
import pickle
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

linreg1 = pickle.load(open('write_data/lr_model.pkl', 'rb'))

In [55]:
data = pd.read_csv('raw_data/advertising.csv')
data = data.drop('Unnamed: 0',axis=1)
data.describe()


,TV,radio,newspaper,sales
count,200.000000,200.000000,200.000000,200.000000
mean,147.042500,23.264000,30.554000,14.022500
std,85.854236,14.846809,21.778621,5.217457
min,0.700000,0.000000,0.300000,1.600000
25%,74.375000,9.975000,12.750000,10.375000
50%,149.750000,22.900000,25.750000,12.900000
75%,218.825000,36.525000,45.100000,17.400000
max,296.400000,49.600000,114.000000,27.000000


In [56]:
X = data.drop('sales', axis=1)
y = data['sales']

In [57]:
# split the data into training and testing set
X_train , X_test, y_train, y_test = train_test_split(X, y,random_state=2019)

### 1. Write a Function to calculate train and test error for different polynomial degree (1-9)? 

In [59]:
def cal_degree(poly_degree):
    '''calculate train and test error for different polynomial degree (1-9)'''
    train_errors = None
    test_errors = None
    
#       For each model with different polynomial degree:
#       Create object for PolynomialFeatures with necessary parameters
#       get the polynomialfeatures for training and testing
#       Fit the LinearRegression to new Features and y_train
#       Calculate the testing and training error (Root mean squared error)
#       Add train and test errors to numpy arrays
        
    return train_errors, test_errors

In [22]:
error_train = cal_degree(10)[0].tolist()
error_test = cal_degree(10)[1].tolist()

#error_train = [1.6872219375656985, 0.6412867668294241, 0.4711637753473785, 0.36944896135164984, 
                0.2410561613114944, 0.2564919445411836, 0.3165437580993593,
                0.3903584583915307, 0.32526008157451436, 0.7955206228171482]
#error_test = [1.6340815294866986, 0.4951157689460942, 0.4798020916835737, 0.4870596445796682,
               0.4608682794261582, 1.198082221301244, 4.440474842230711, 
               19.919529327003232, 129.4605767258836, 1066.1109145234598]

### 2. How does increasing polynomial degree effect Bias and Variance? 

![rsme](visuals/rsme_poly.png)

// your answer here //

### 3. What method would you use to mitigate the overfitting and underfitting? 

// your answer here //

### 4. Find the optimal alpha value for Lasso Regression for the given polynomial features

In [49]:
X_train , X_test, y_train, y_test = train_test_split(X, y,random_state=2019)

poly = PolynomialFeatures(degree=10, interaction_only=False, include_bias=False)

X_poly_train = poly.fit_transform(X_train)
X_poly_test = poly.transform(X_test)


for i, alpha in enumerate(np.linspace(.1, 10, SIZE)):
    lasso = Lasso(alpha=alpha, tol=0.06)
    lasso.fit(X_poly_train,y_train)
    y_pred = lasso.predict(X_poly_test)
    score = mean_squared_error(y_pred,y_test)
    error[i] = score

alphas[np.argmin(error)]

0.10101010101010102